![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [17]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


#### Mapping target values into categorical variables

In [19]:
cc_apps[13] = cc_apps[13].map(lambda x:1 if x == '+' else 0)
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,1
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,1


#### Checking for missing values

In [21]:
cc_apps.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64

#### Checking if data types are correct

In [23]:
cc_apps.dtypes

0      object
1      object
2     float64
3      object
4      object
5      object
6      object
7     float64
8      object
9      object
10      int64
11     object
12      int64
13      int64
dtype: object

#### Investigate why column 1 is object type

In [25]:
cc_apps[1]

0      30.83
1      58.67
2      24.50
3      27.83
4      20.17
       ...  
685    21.08
686    22.67
687    25.25
688    17.92
689    35.00
Name: 1, Length: 690, dtype: object

In [26]:
(cc_apps[1]=="?").sum()

12

In [27]:
cc_apps = cc_apps[cc_apps[1] != "?"].reset_index()

In [28]:
cc_apps[1] = cc_apps[1].astype("float")

#### Converting categorical variables into dummy variables

In [30]:
df_cat = pd.get_dummies(cc_apps[[0, 3, 4 ,5, 6, 8, 9, 11]], drop_first=True)
print(df_cat.shape)
df_cat

(678, 35)


,0_a,0_b,3_l,3_u,3_y,4_g,4_gg,4_p,5_aa,5_c,...,6_h,6_j,6_n,6_o,6_v,6_z,8_t,9_t,11_p,11_s
0,False,True,False,True,False,True,False,False,False,False,...,False,False,False,False,True,False,True,True,False,False
1,True,False,False,True,False,True,False,False,False,False,...,True,False,False,False,False,False,True,True,False,False
2,True,False,False,True,False,True,False,False,False,False,...,True,False,False,False,False,False,True,False,False,False
3,False,True,False,True,False,True,False,False,False,False,...,False,False,False,False,True,False,True,True,False,False
4,False,True,False,True,False,True,False,False,False,False,...,False,False,False,False,True,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,False,True,False,False,True,False,False,True,False,False,...,True,False,False,False,False,False,False,False,False,False
674,True,False,False,True,False,True,False,False,False,True,...,False,False,False,False,True,False,False,True,False,False
675,True,False,False,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
676,False,True,False,True,False,True,False,False,True,False,...,False,False,False,False,True,False,False,False,False,False


#### Scaling numerical variables

In [32]:
scaler = StandardScaler()
df_num =cc_apps[[1,2,7,10,12]]
df_num_scaled = pd.DataFrame(scaler.fit_transform(df_num))
print(df_num_scaled.shape)


(678, 5)


#### Final formatted dataframe

In [34]:
final_df = pd.concat([df_cat, df_num_scaled, cc_apps[13]], axis=1, ignore_index=True)
print(final_df.shape)
final_df

(678, 41)


,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,False,True,False,True,False,True,False,False,False,False,...,True,True,False,False,-0.061777,-0.956759,-0.286483,-0.293276,-0.194593,1
1,True,False,False,True,False,True,False,False,False,False,...,True,True,False,False,2.268118,-0.063607,0.248119,0.728518,-0.087887,1
2,True,False,False,True,False,True,False,False,False,False,...,True,False,False,False,-0.591526,-0.856630,-0.211818,-0.497635,-0.037583,1
3,False,True,False,True,False,True,False,False,False,False,...,True,True,False,False,-0.312843,-0.648361,0.460168,0.524159,-0.194021,1
4,False,True,False,True,False,True,False,False,False,False,...,True,False,False,True,-0.953898,0.169694,-0.149099,-0.497635,-0.194593,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,False,True,False,False,True,False,False,True,False,False,...,False,False,False,False,-0.877742,1.062845,-0.286483,-0.497635,-0.194593,0
674,True,False,False,True,False,True,False,False,False,True,...,False,True,False,False,-0.744677,-0.806565,-0.062487,-0.088917,-0.119518,0
675,True,False,False,False,True,False,False,True,False,False,...,False,True,False,False,-0.528760,1.746727,-0.062487,-0.293276,-0.194402,0
676,False,True,False,True,False,True,False,False,True,False,...,False,False,False,False,-1.142198,-0.915706,-0.647862,-0.497635,-0.051684,0


#### Splitting data

In [36]:
X = final_df.iloc[:, :-1].copy()
print(X.shape)
X.head()

(678, 40)


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,False,True,False,True,False,True,False,False,False,False,...,False,True,True,False,False,-0.061777,-0.956759,-0.286483,-0.293276,-0.194593
1,True,False,False,True,False,True,False,False,False,False,...,False,True,True,False,False,2.268118,-0.063607,0.248119,0.728518,-0.087887
2,True,False,False,True,False,True,False,False,False,False,...,False,True,False,False,False,-0.591526,-0.856630,-0.211818,-0.497635,-0.037583
3,False,True,False,True,False,True,False,False,False,False,...,False,True,True,False,False,-0.312843,-0.648361,0.460168,0.524159,-0.194021
4,False,True,False,True,False,True,False,False,False,False,...,False,True,False,False,True,-0.953898,0.169694,-0.149099,-0.497635,-0.194593


In [37]:
y = final_df[40].copy()
print(y.shape)
y.head()

(678,)


0    1
1    1
2    1
3    1
4    1
Name: 40, dtype: int64

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(474, 40)
(474,)
(204, 40)
(204,)


#### Training and evaluating Logistic Regression model

In [40]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [41]:
confusion_matrix(y_test, model.predict(X_test))

array([[91, 19],
       [ 9, 85]], dtype=int64)

In [42]:
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.91      0.83      0.87       110
           1       0.82      0.90      0.86        94

    accuracy                           0.86       204
   macro avg       0.86      0.87      0.86       204
weighted avg       0.87      0.86      0.86       204



In [62]:
best_score = accuracy_score(y_test, model.predict(X_test))
best_score

0.8627450980392157

#### GridSearch for optimal C parameter

In [71]:
log = LogisticRegression()
clf = GridSearchCV(log, {'C' : [10**x for x in range(-2, 5)]})
clf.fit(X_train, y_train)
                   

C:\Users\danie\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\danie\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.1, 1, 10, 100, 1000, 10000]})

In [83]:
clf.best_params_

{'C': 1}

#### Logistic Regression achived best accuracy with default C = 1, which was the default parameter.